*<a id="topo"></a>
___
<img src="../_images/logo_mei.jpg" alt="Mestrado em Internet das Coisas @ IPT" width="200"/>
<div class="alert alert-block alert-success" align="center">
<h1>Análise e Processamento de Grandes Volumes de Dados</h1>
<h3>Sentiment analysis - 4ª parte</div>
<center><h5>Criado por: Bruno Bernardo / David Carrilho / Rui Rodrigues</h5></center>
___

[<img src="../_images/download.jpg" alt="Mestrado em Internet das Coisas @ IPT" width="50"/>](lstm.ipynb)
___

**Crédito para Peter Nagy February 2017** 
https://github.com/nagypeterjob

[Análise sentimental supervisionada (Naive Bayes)](#super)<br>
[Preparação](#preparacao)<br>
[Importação](#import)<br>
[Dataset](#dataset)<br>
[Importação dos dados](#import2)<br>
[Separação de dados em treino e teste](#split)<br>
[Visualização dos dados de treino](#visual)<br>
[Preparação subset de treino](#treino)<br>
[Extração de características](#features)<br>
[Classificação](#classificacao)<br>
[Conclusão](#conclusao)

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

<a id="super"></a>
# Análise sentimental supervisionada (LSTM)
<p>&nbsp;</p>

Depois de termos demonstrado como obter análise de sentimento sobre ficheiros e <i>feeds</i> usando uma API vamos agora demonstrar como usar directamente o [NLTK](#http://www.nltk.org/) (<i>Natural Language Tool Kit</i>) para treinar classificadores estatísticos e posteriormente usá-los para fazer a análise sentimental. Para isto usamos datasets recolhidos de [CrowdFlower](#https://www.crowdflower.com/data-for-everyone/). Foram escolhidos dois datasets com apenas 3 classes mas este método permite usa mais classes.

Neste primeiro exemplo vamos usar o classificador NB. Este classificador assume que as características dos vectores são independentes. Quando esta assumpção se verifica este é um classificador de alta precisão. É o que vamos avaliar.


------

<a id="preparacao"></a>
## Preparação

Se correr o notebook no seu próprio server Jupyter deverá instalar dependências.

Na "Anaconda Comand Prompt" (na pasta onde tens os notebooks):

    pip install numpy

    pip install pandas

    pip install nltk

    pip install wordcloud
    
    
<div class="alert alert-block alert-info">NOTA: Se não tiveres premissões para instalar, executa o Prompt como Administrador. Se estiveres da drive C: e os notebooks estiverem na D:, assim que abres o Prompt deves escrever D: para mudar de drive.</div>

<div class="alert alert-block alert-danger">Dependendo da vossa versão do Python poderão ter dificuldade em instalar as dependências do Wordcloud. Se for esse o caso façam o download da versão adequada [daqui](#https://www.lfd.uci.edu/%7Egohlke/pythonlibs/#wordcloud) e executem:
    pip install XXXXX sendo XXXXX o nome do ficheiro descarregado.</div>    

No nosso caso vamos usar um contentor dinâmico que permite a partilha de notebooks com edição, [mybinder.org](https://mybinder.org).

<a id="import"></a>
### A importação dos módulos necessários

In [7]:
# algebra linear
import numpy as np 
# processamento de dados e I/O ficheiros 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

<a id="dataset"></a>
### O Dataset

Preparámos dois datasets:

    * Judge emotions about nuclear energy from Twitter (2013)

    É uma colecção de tweets relacionados com energia nuclear juntos com a classificação feita por utilizadores do sentimento relacionado com o tweet. As categorias disponíveis são: "Positivo", "Neutro" e "Negativo". O dataset contém ainda informação sobre o nível de confianção numa categorização correcta.
    
    * First GOP debate sentiment analysis (2015)

    É uma colecção de dezenas de milhares de tweets sobre o debate dos candidatos presidenciais republicanos dos EUA. Aos utilizadores foi pedido que categorizassem a nível de sentimento, a relevância, candidato a que se refere e nível de confiança na categorização correcta.

Foram escolhidos estes dois precisamente por usarem apenas 3 classes, o que se adequa à nossa demonstração. 


<a id="import2"></a>
### Importação dos dados

Os nossos <i>dataset</i> são CSVs que contém diversa informação, como o nosso objectivo é fazer análise sentimental precisamos apenas das colunas com o texto do tweet e a classificação atribuida por utilizadores. 

In [6]:
data = pd.read_csv('../_datasets/GOP_REL_ONLY.csv', encoding = "ISO-8859-1")
#data = pd.read_csv('../_datasets/1377191648_sentiment_nuclear_power.csv', encoding = "ISO-8859-1")
# Keeping only the neccessary columns
data = data[['text','sentiment']]

Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [3]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
16986


Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(rate=0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 28, 128)       256000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 196)           254800      embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             394         lstm_1[0][0]                     
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
____________________________________________________________________________________________________
None


Hereby I declare the train and test dataset.

In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
57s - loss: 0.4678 - acc: 0.8066
Epoch 2/7
56s - loss: 0.3480 - acc: 0.8529
Epoch 3/7
55s - loss: 0.2970 - acc: 0.8740
Epoch 4/7
56s - loss: 0.2718 - acc: 0.8848
Epoch 5/7
56s - loss: 0.2459 - acc: 0.8965
Epoch 6/7
56s - loss: 0.2309 - acc: 0.9008
Epoch 7/7
56s - loss: 0.2129 - acc: 0.9060


Extracting a validation set, and measuring score and accuracy.

In [7]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Indicadores de performance:")
print("score: %.2f%%" % (score*100))
print("acc: %.2f%%" % (acc*100))

score: 0.42
acc: 0.84


Finally measuring the number of correct guesses.  It is clear that finding negative tweets goes very well for the Network but deciding whether is positive is not really. My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

In [8]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print('[Negative]: %0.2f%%'  % (neg_correct/neg_cnt*100))       
print('[Positive]: %0.2f%%'  % (pos_correct/pos_cnt*100))


pos_acc 56.63430420711975 %
neg_acc 92.7791771620487 %
